In [7]:
import numpy as np
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
from pathlib import PurePath
import cv2

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import keras
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Input, BatchNormalization
from keras.optimizers import SGD, Adam
from keras import regularizers
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Add


In [8]:


number_images_train = np.load('../getDataSet/dataset/images_train.npy')
number_labels_train = np.load('../getDataSet/dataset/labels_train.npy')[:,0]
number_images_test = np.load('../getDataSet/dataset/images_test.npy')
number_labels_test = np.load('../getDataSet/dataset/labels_test.npy')[:,0]
print(Counter(number_labels_train))
print(Counter(number_labels_test))

le = LabelEncoder()
le.fit(number_labels_train)
number_labels_train = le.transform(number_labels_train)
number_labels_test = le.transform(number_labels_test)

Counter({'K': 379, 'A': 374, '5': 374, '3': 373, 'J': 373, 'Q': 372, '6': 371, 'T': 370, '2': 367, '4': 366, '9': 357, '8': 357, '7': 351})
Counter({'7': 108, '8': 105, '9': 101, '4': 95, '2': 95, '6': 91, 'T': 90, '3': 87, 'J': 87, 'Q': 86, 'A': 85, '5': 85, 'K': 81})


In [9]:
# Function to create a residual block
def residual_block(x, filters, kernel_size=3, stride=1, activation='relu'):
    shortcut = x
    x = Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    x = Add()([x, shortcut])
    return x

# Create the model
input_shape = (70, 70, 1)
inputs = Input(shape=input_shape)

x = Conv2D(32, (3, 3), padding='same', activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.1)(x)

x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.1)(x)

# Add the first residual block
x = residual_block(x, 64)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.1)(x)

# Add the second residual block
x = residual_block(x, 64)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.1)(x)

x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dense(13, activation='softmax')(x)

numbermodel = Model(inputs=inputs, outputs=x)

# Compile the model
opt = keras.optimizers.Adam()
numbermodel.compile(optimizer=opt,
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

numbermodel.summary()

# # Create the Sequential model
# numbermodel = Sequential()

# # Add input layer
# numbermodel.add(Input(shape=(70, 70, 1)))

# # Add convolutional, batch normalization, pooling, and dropout layers
# numbermodel.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
# numbermodel.add(BatchNormalization())
# numbermodel.add(MaxPooling2D((2, 2)))
# numbermodel.add(Dropout(0.3))

# numbermodel.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
# numbermodel.add(BatchNormalization())
# numbermodel.add(MaxPooling2D((2, 2)))
# numbermodel.add(Dropout(0.3))

# numbermodel.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
# numbermodel.add(BatchNormalization())
# numbermodel.add(MaxPooling2D((2, 2)))
# numbermodel.add(Dropout(0.3))

# numbermodel.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
# numbermodel.add(BatchNormalization())
# numbermodel.add(MaxPooling2D((2, 2)))
# numbermodel.add(Dropout(0.4))

# # Flatten the output and add dense layers
# numbermodel.add(Flatten())
# numbermodel.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
# numbermodel.add(Dense(13, activation='softmax'))

# # Compile the model
# opt = keras.optimizers.Adam(learning_rate=0.001)
# numbermodel.compile(optimizer=opt,
#                     loss='sparse_categorical_crossentropy',
#                     metrics=['accuracy'])

# # Print the model summary
# numbermodel.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 70, 70, 1)]          0         []                            
                                                                                                  
 conv2d_4 (Conv2D)           (None, 70, 70, 32)           320       ['input_2[0][0]']             
                                                                                                  
 max_pooling2d_4 (MaxPoolin  (None, 35, 35, 32)           0         ['conv2d_4[0][0]']            
 g2D)                                                                                             
                                                                                                  
 dropout_4 (Dropout)         (None, 35, 35, 32)           0         ['max_pooling2d_4[0][0]'

In [5]:
numbermodel.fit(number_images_train, number_labels_train,epochs=50, validation_data=(number_images_test, number_labels_test),batch_size=64)

Epoch 1/50


75/75 [==============================] - 12s 128ms/step - loss: 2.7456 - accuracy: 0.1089 - val_loss: 2.5598 - val_accuracy: 0.0903
Epoch 2/50
75/75 [==============================] - 10s 127ms/step - loss: 2.4200 - accuracy: 0.1384 - val_loss: 2.5703 - val_accuracy: 0.0711
Epoch 3/50
75/75 [==============================] - 10s 133ms/step - loss: 2.2184 - accuracy: 0.2084 - val_loss: 2.5064 - val_accuracy: 0.1095
Epoch 4/50
75/75 [==============================] - 9s 125ms/step - loss: 1.9555 - accuracy: 0.2993 - val_loss: 2.5582 - val_accuracy: 0.1254
Epoch 5/50
75/75 [==============================] - 10s 130ms/step - loss: 1.5948 - accuracy: 0.4264 - val_loss: 2.3084 - val_accuracy: 0.1848
Epoch 6/50
75/75 [==============================] - 10s 130ms/step - loss: 1.2704 - accuracy: 0.5299 - val_loss: 2.1442 - val_accuracy: 0.1781
Epoch 7/50
75/75 [==============================] - 11s 142ms/step - loss: 0.8949 - accuracy: 0.6946 - val_loss: 2.9265 - val_accuracy: 0.168

In [6]:
numbermodel.save("./models/NumberResCNN.keras")
np.save('./encoders/NumberEncoder.npy', le.classes_)

# encoder = LabelEncoder()
# encoder.classes_ =  np.load('classes.npy')